# Interactive Dashboard

# PhDs Awarded in the US

By: Dapo Adegbile

### In this blog post I will be examining how many and the types of Doctorates given out in the United States in 2017. The data is originally taken from: https://ncses.nsf.gov/pubs/nsf19301/data. I cleaned the dataset and perfomed some analysis using `pandas` , and I then made a dashboard to better visualize key insights from the data in plotly and Streamlit. 

### I chose to use Plotly as I am already familar with it's visualizations and I choose to use streamlit to deploy the dashboard because of the ease of use and the sleak display of the dashboard.


## Data Cleaning

### My data cleaning process can be found below:

In [1]:
#Collapse
import pandas as pd
import numpy as np

In [2]:
#Collapse
state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]


In [68]:
#Collapse
df = pd.read_excel("/Users/dapoadegbile/Documents/data_tables/sed17-sr-tab007.xlsx", header = 4)


/Users/dapoadegbile/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:214: UserWarning:

Workbook contains no default style, apply openpyxl's default



In [5]:
#Collapse
df1 = df.set_index(df.iloc[:,0])


In [7]:
#Collapse
df1.iloc[:,0].value_counts

<bound method IndexOpsMixin.value_counts of Unnamed: 0
NaN                                          NaN
All institutions                All institutions
Alabama                                  Alabama
Alabama A&M U.                    Alabama A&M U.
Alabama State U.                Alabama State U.
                                   ...          
Medical C. Wisconsin        Medical C. Wisconsin
U. Wisconsin-Madison        U. Wisconsin-Madison
U. Wisconsin-Milwaukee    U. Wisconsin-Milwaukee
Wyoming                                  Wyoming
U. Wyoming                            U. Wyoming
Name: Unnamed: 0, Length: 482, dtype: object>

In [8]:
#Collapse
df1 = df1.loc[state_names]


,Unnamed: 0,Unnamed: 1,Life sciences,Unnamed: 3,Unnamed: 4,Unnamed: 5,Physical sciences and earth sciences,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Alaska,Alaska,52.0,10.0,4.0,6.0,0.0,20.0,2.0,15.0,3.0,...,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
Alabama,Alabama,707.0,188.0,32.0,122.0,34.0,59.0,23.0,7.0,29.0,...,131.0,5.0,4.0,14.0,17.0,22.0,6.0,13.0,18.0,32.0
Arkansas,Arkansas,202.0,75.0,18.0,35.0,22.0,17.0,11.0,1.0,5.0,...,33.0,0.0,4.0,2.0,4.0,7.0,4.0,2.0,1.0,9.0
Arizona,Arizona,801.0,133.0,23.0,74.0,36.0,110.0,31.0,29.0,50.0,...,141.0,2.0,10.0,8.0,10.0,50.0,6.0,12.0,12.0,31.0
California,California,6105.0,1300.0,111.0,1015.0,174.0,841.0,367.0,178.0,296.0,...,1151.0,38.0,166.0,97.0,57.0,269.0,10.0,164.0,153.0,197.0
Colorado,Colorado,1005.0,212.0,25.0,162.0,25.0,155.0,42.0,47.0,66.0,...,206.0,22.0,8.0,14.0,39.0,29.0,0.0,12.0,29.0,53.0
Connecticut,Connecticut,748.0,196.0,14.0,155.0,27.0,104.0,56.0,15.0,33.0,...,97.0,0.0,18.0,12.0,3.0,17.0,0.0,10.0,18.0,19.0
District of Columbia,District of Columbia,623.0,55.0,0.0,50.0,5.0,37.0,19.0,6.0,12.0,...,78.0,0.0,3.0,0.0,4.0,5.0,0.0,1.0,10.0,55.0
Delaware,Delaware,238.0,59.0,9.0,40.0,10.0,24.0,2.0,11.0,11.0,...,69.0,0.0,0.0,27.0,13.0,7.0,0.0,11.0,8.0,3.0


In [9]:
#Collapse
df1.reset_index(drop = True, inplace = True)


,Unnamed: 0,Unnamed: 1,Life sciences,Unnamed: 3,Unnamed: 4,Unnamed: 5,Physical sciences and earth sciences,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,Alaska,52.0,10.0,4.0,6.0,0.0,20.0,2.0,15.0,3.0,...,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
1,Alabama,707.0,188.0,32.0,122.0,34.0,59.0,23.0,7.0,29.0,...,131.0,5.0,4.0,14.0,17.0,22.0,6.0,13.0,18.0,32.0
2,Arkansas,202.0,75.0,18.0,35.0,22.0,17.0,11.0,1.0,5.0,...,33.0,0.0,4.0,2.0,4.0,7.0,4.0,2.0,1.0,9.0
3,Arizona,801.0,133.0,23.0,74.0,36.0,110.0,31.0,29.0,50.0,...,141.0,2.0,10.0,8.0,10.0,50.0,6.0,12.0,12.0,31.0
4,California,6105.0,1300.0,111.0,1015.0,174.0,841.0,367.0,178.0,296.0,...,1151.0,38.0,166.0,97.0,57.0,269.0,10.0,164.0,153.0,197.0
5,Colorado,1005.0,212.0,25.0,162.0,25.0,155.0,42.0,47.0,66.0,...,206.0,22.0,8.0,14.0,39.0,29.0,0.0,12.0,29.0,53.0
6,Connecticut,748.0,196.0,14.0,155.0,27.0,104.0,56.0,15.0,33.0,...,97.0,0.0,18.0,12.0,3.0,17.0,0.0,10.0,18.0,19.0
7,District of Columbia,623.0,55.0,0.0,50.0,5.0,37.0,19.0,6.0,12.0,...,78.0,0.0,3.0,0.0,4.0,5.0,0.0,1.0,10.0,55.0
8,Delaware,238.0,59.0,9.0,40.0,10.0,24.0,2.0,11.0,11.0,...,69.0,0.0,0.0,27.0,13.0,7.0,0.0,11.0,8.0,3.0
9,Florida,2336.0,490.0,77.0,285.0,128.0,259.0,94.0,64.0,101.0,...,419.0,21.0,26.0,29.0,45.0,78.0,23.0,48.0,51.0,98.0


In [10]:
# df1[["State or location and institution", "All fields", "Science", "Engineering"]]


Index(['Unnamed: 0', 'Unnamed: 1', 'Life sciences', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Physical sciences and earth sciences', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Mathematics and computer sciences',
       'Unnamed: 11', 'Unnamed: 12', 'Psychology and social sciences ',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29'],
      dtype='object')

In [11]:
#Collapse

df_final = df1[['Unnamed: 0', 'Unnamed: 1', 'Life sciences', 
     'Physical sciences and earth sciences',
     'Mathematics and computer sciences', 'Psychology and social sciences ', 'Unnamed: 20' ]]


,Unnamed: 0,Unnamed: 1,Life sciences,Physical sciences and earth sciences,Mathematics and computer sciences,Psychology and social sciences,Unnamed: 20
0,Alaska,52.0,10.0,20.0,0.0,17.0,3.0
1,Alabama,707.0,188.0,59.0,48.0,89.0,131.0
2,Arkansas,202.0,75.0,17.0,10.0,19.0,33.0
3,Arizona,801.0,133.0,110.0,51.0,156.0,141.0
4,California,6105.0,1300.0,841.0,495.0,1132.0,1151.0


In [12]:
#Collapse

df_final.columns = ['State', "All Fields" , 'Life Sciences' ,
                   'Physical Sciences and Earth Sciences' , 'Mathematics and Computer Sciences' ,
                    'Psychology and Social Sciences' , 'Engineering']


,State,All Fields,Life Sciences,Physical Sciences and Earth Sciences,Mathematics and Computer Sciences,Psychology and Social Sciences,Engineering
0,Alaska,52.0,10.0,20.0,0.0,17.0,3.0
1,Alabama,707.0,188.0,59.0,48.0,89.0,131.0
2,Arkansas,202.0,75.0,17.0,10.0,19.0,33.0
3,Arizona,801.0,133.0,110.0,51.0,156.0,141.0
4,California,6105.0,1300.0,841.0,495.0,1132.0,1151.0
5,Colorado,1005.0,212.0,155.0,67.0,128.0,206.0
6,Connecticut,748.0,196.0,104.0,60.0,129.0,97.0
7,District of Columbia,623.0,55.0,37.0,24.0,207.0,78.0
8,Delaware,238.0,59.0,24.0,19.0,45.0,69.0
9,Florida,2336.0,490.0,259.0,145.0,409.0,419.0


In [69]:
# Collapse
df_exports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

df_codes = df_final.merge(df_exports, left_on = 'State', right_on = 'state', how = 'left')
df_codes.head()

,State,All Fields,Life Sciences,Physical Sciences and Earth Sciences,Mathematics and Computer Sciences,Psychology and Social Sciences,Engineering,code,state,category,...,dairy,fruits fresh,fruits proc,total fruits,veggies fresh,veggies proc,total veggies,corn,wheat,cotton
0,Alaska,52.0,10.0,20.0,0.0,17.0,3.0,AK,Alaska,state,...,0.19,0.0,0.0,0.00,0.6,1.0,1.56,0.0,0.0,0.00
1,Alabama,707.0,188.0,59.0,48.0,89.0,131.0,AL,Alabama,state,...,4.06,8.0,17.1,25.11,5.5,8.9,14.33,34.9,70.0,317.61
2,Arkansas,202.0,75.0,17.0,10.0,19.0,33.0,AR,Arkansas,state,...,3.53,2.2,4.7,6.88,4.4,7.1,11.45,69.5,114.5,665.44
3,Arizona,801.0,133.0,110.0,51.0,156.0,141.0,AZ,Arizona,state,...,105.48,19.3,41.0,60.27,147.5,239.4,386.91,7.3,48.7,423.95
4,California,6105.0,1300.0,841.0,495.0,1132.0,1151.0,CA,California,state,...,929.95,2791.8,5944.6,8736.40,803.2,1303.5,2106.79,34.6,249.3,1064.95


In [23]:
#Collapse
df_codes = df_codes.iloc[:,0:8]

df_codes.to_csv('/Users/dapoadegbile/Documents/Biostat 823/Homework/DoctoratesbyStateandDegree.csv', index_label = False)

,State,All Fields,Life Sciences,Physical Sciences and Earth Sciences,Mathematics and Computer Sciences,Psychology and Social Sciences,Engineering,code
0,Alaska,52.0,10.0,20.0,0.0,17.0,3.0,AK
1,Alabama,707.0,188.0,59.0,48.0,89.0,131.0,AL
2,Arkansas,202.0,75.0,17.0,10.0,19.0,33.0,AR
3,Arizona,801.0,133.0,110.0,51.0,156.0,141.0,AZ
4,California,6105.0,1300.0,841.0,495.0,1132.0,1151.0,CA


### Below are the plots used the in dashboard and the code that accompanies it.  Using a choropleth map, we can cleanly see how the popularity of each field varies from state to state. The total number of schools is represented by the color gradient, and you can filter the fields to see which fields are more popular within each state. Additionally I added a bar chart that shows each fields popularity in each stat as another means of comparison. 

### From the dashboard we can see that California gave out the most doctorate degrees in 2017, followed by Texas and New York. Additionally LIfe Sciences is the hottest field, while Engineering and Psychology and Social Sciences are a close second and third. 

In [39]:
import plotly.graph_objects as go

for col in df_codes.columns:
    df_codes[col] = df_codes[col].astype(str)

# df_codes['text'] = df_codes['All Fields'] + '<br>' + \
#     'Life Sciences ' + df_codes['Life Sciences'] + ' Physical Sciences and Earth Sciences ' + df_codes['Physical Sciences and Earth Sciences'] + '<br>' + \
#     'Mathematics and Computer Sciences ' + df_codes['Mathematics and Computer Sciences'] + ' Psychology and Social Sciences ' + df_codes['Psychology and Social Sciences'] + '<br>' + \
#     'Engineering ' + df_codes['Engineering']




fig = go.Figure(data=go.Choropleth(
    locations= df_codes['code'], # Spatial coordinates
    z = df_codes['All Fields'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Greens',
#     hover_name = "State", hover_data = "All Fields",
    text=df_codes['State'], # hover text
    colorbar_title = "Number of Doctorates",
))

fig.update_layout(
    title_text = 'Number of Doctorates by State',
    geo_scope='usa', # limit map scope to USA
)

fig.show()

In [67]:
fig = go.Figure(go.Bar(
            y=(df_codes['All Fields']).astype(float),
            x= df_codes['State'],
            marker_color = 'indianred'))
fig.update_layout(xaxis_tickangle=-90)

fig.show()